In [ ]:
"""
Created on Mon Mar 06 17:05 2023

This script is to have a look at the area of the largest ice shelves in both ensembles

@author: Clara Burgard
"""

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import basal_melt_param.useful_functions as uf
import basal_melt_param.melt_functions as meltf
from basal_melt_param.constants import *
import basal_melt_param.T_S_profile_functions as tspf
import seaborn as sns
from matplotlib import cm
import cartopy
import cartopy.crs as ccrs
import matplotlib as mpl
import cmocean
import glob

In [ ]:
%matplotlib qt5

FIXED MASK FILE FOR OPM016

In [ ]:
inputpath_mask = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/ANTARCTICA_IS_MASKS/nemo_5km_OPM016/'
file_isf_mathiot_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_new_oneFRIS.nc')
#file_isf = file_isf_orig.sel(Nisf=isf_list)
nonnan_Nisf = file_isf_mathiot_orig['Nisf'].where(np.isfinite(file_isf_mathiot_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_mathiot_orig.sel(Nisf=nonnan_Nisf)
large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
file_isf_mathiot = file_isf_nonnan.sel(Nisf=large_isf)
area_mathiot = file_isf_mathiot['isf_area_here']

In [ ]:
nemo_run = 'bi646'
inputpath_mask='/bettik/burgardc/DATA/NN_PARAM/interim/ANTARCTICA_IS_MASKS/SMITH_'+nemo_run+'/'
file_isf_1970 = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_oneFRIS_2041.nc').drop('time')

In [ ]:
file_isf_1970['dIF'].plot()

In [ ]:
new_kisf_list = []
for kisf in file_isf_mathiot.Nisf:
    if file_isf_mathiot['isf_name'].sel(Nisf=kisf).values in file_isf_1970['isf_name'].values:
        new_kisf = file_isf_1970.Nisf.where(file_isf_1970['isf_name'] == file_isf_mathiot['isf_name'].sel(Nisf=kisf), drop=True).values
    elif file_isf_mathiot['isf_name'].sel(Nisf=kisf).values == 'Ekstrom':
        new_kisf = np.array([110])
    elif file_isf_mathiot['isf_name'].sel(Nisf=kisf).values == 'Tracy Tremenchus':
        new_kisf = np.array([23])
    else:
        print(file_isf_mathiot['isf_name'].sel(Nisf=kisf).values)
    new_kisf_list.append(new_kisf)
        
new_kisf_array = np.concatenate(new_kisf_list, axis=0)

In [ ]:
area_mathiot['Nisf'] = new_kisf_array.astype(int)

In [ ]:
nemo_run = 'bi646'
inputpath_mask='/bettik/burgardc/DATA/NN_PARAM/interim/ANTARCTICA_IS_MASKS/SMITH_'+nemo_run+'/'

area_list = []
for tt in range(1970, 1970 + 72):
    file_isf_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_oneFRIS_'+str(tt)+'.nc').drop('time')
    file_isf = file_isf_orig
    #nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
    #file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
    #file_isf = file_isf_nonnan.sel(Nisf=nonnan_Nisf)
    #large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
    #file_isf = file_isf_nonnan.sel(Nisf=large_isf)
    if 'labels' in file_isf.coords.keys():
        file_isf = file_isf.drop('labels')
    area_list.append(file_isf['isf_area_here'])
    
areas_bi646 = xr.concat(area_list, dim='time')

In [ ]:
nemo_run = 'bf663'
inputpath_mask='/bettik/burgardc/DATA/NN_PARAM/interim/ANTARCTICA_IS_MASKS/SMITH_'+nemo_run+'/'

area_list = []
for tt in range(1970, 2051):
    file_isf_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_oneFRIS_'+str(tt)+'.nc').drop('time')
    file_isf = file_isf_orig
    #nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
    #file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
    #file_isf = file_isf_orig.sel(Nisf=nonnan_Nisf)
    #large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
    #file_isf = file_isf_nonnan.sel(Nisf=large_isf)
    if 'labels' in file_isf.coords.keys():
        file_isf = file_isf.drop('labels')
    area_list.append(file_isf['isf_area_here'])
    
areas_bf663 = xr.concat(area_list, dim='time')

In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','brown','magenta','yellow','cyan','blue','magenta']

f = plt.figure()
f.set_size_inches(8.25*1.5, 8.25*1.5)

ax={}

leg_hdl = []

i = 0

sorted_isf_all = [11,69,43,12,70,44,29,13,58,71,45,30,31,61,73,47,32,48,33,17,62,49,34,18,10,65,51,22,38,52,23,66,53,39,24,40,54,75,25,26,42,55]

for kisf in tqdm(sorted_isf_all):
    
        
    ax[i] = f.add_subplot(7,7,i+1)

    if kisf in areas_bi646.Nisf.values:

        ax[i].plot(areas_bi646.time,areas_bi646.sel(Nisf=kisf), color='red')

    if kisf in areas_bf663.Nisf.values:

        ax[i].plot(areas_bf663.time,areas_bf663.sel(Nisf=kisf), color='blue')

    if kisf in area_mathiot.Nisf.values:

        ax[i].axhline(xmin=0, xmax=80, y=area_mathiot.sel(Nisf=kisf), color='k', linewidth=2)


    #if kisf == 23:
    #    ax[i].set_title('Tracy Tremenchus')
    #elif kisf == 24:
    #    ax[i].set_title('Conger/Glenzer')
    #elif kisf == 110:
    #    ax[i].set_title('Ekström')
    #else:
    ax[i].set_title(str(file_isf_1970['isf_name'].sel(Nisf=kisf).values))
    #ax[i].set_xlim(-2.2,2.5)

    i = i+1


f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()

In [ ]:
str(file_isf_1970['isf_name'].sel(Nisf=kisf).values)

In [ ]:
areas_bf663.Nisf.values